In [ ]:
"""
Test Georgia files
"""

In [30]:
import os
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

In [21]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
inputpath = '/data/cburgard/FOR_GEORGIA/NEMO_1degree/'

In [4]:
data_T = xr.open_dataset(inputpath + 'n42openc_00910101_01001231_1Y_grid_T.nc')

In [6]:
data_T_timmean = data_T.mean('time_counter')

In [10]:
SST = data_T_timmean['thetao'].isel(deptht=0)

In [11]:
lon = SST.nav_lon
lat = SST.nav_lat

In [58]:
plt.figure()
ax = plt.subplot(1,1,1,projection=ccrs.SouthPolarStereo(central_longitude=0)) #
abso0 = ax.pcolormesh(lon,lat,SST,transform=ccrs.PlateCarree(central_longitude=0)) #,vmin=0,vmax=30
ax.coastlines(resolution='110m',linewidth=0.5)
ax.set_extent([-180,180,-90,-60], crs=ccrs.PlateCarree())

In [28]:
SST.plot()

In [31]:
def z_masked_overlap(axe, X, Y, Z, source_projection=None):
    """
    for data in projection axe.projection
    find and mask the overlaps (more 1/2 the axe.projection range)

    X, Y either the coordinates in axe.projection or longitudes latitudes
    Z the data
    operation one of 'pcorlor', 'pcolormesh', 'countour', 'countourf'

    if source_projection is a geodetic CRS data is in geodetic coordinates
    and should first be projected in axe.projection

    X, Y are 2D same dimension as Z for contour and contourf
    same dimension as Z or with an extra row and column for pcolor
    and pcolormesh

    return ptx, pty, Z
    """
    if not hasattr(axe, 'projection'):
        return X, Y, Z

    if not isinstance(axe.projection, ccrs.Projection):
        return X, Y, Z

    if len(X.shape) != 2 or len(Y.shape) != 2:
        return X, Y, Z

    if (source_projection is not None and
        isinstance(source_projection, ccrs.Geodetic)):
        transformed_pts = axe.projection.transform_points(
            source_projection, X, Y)
        ptx, pty = transformed_pts[..., 0], transformed_pts[..., 1]
    else:
        ptx, pty = X, Y

    with np.errstate(invalid='ignore'):
        # diagonals have one less row and one less columns
        diagonal0_lengths = np.hypot(
            ptx[1:, 1:] - ptx[:-1, :-1],
            pty[1:, 1:] - pty[:-1, :-1]
        )

        diagonal1_lengths = np.hypot(
            ptx[1:, :-1] - ptx[:-1, 1:],
            pty[1:, :-1] - pty[:-1, 1:]
        )

        to_mask = (
            (diagonal0_lengths > (
                abs(axe.projection.x_limits[1]
                    - axe.projection.x_limits[0])) / 2) |
            np.isnan(diagonal0_lengths) |
            (diagonal1_lengths > (
                abs(axe.projection.x_limits[1]
                    - axe.projection.x_limits[0])) / 2) |
            np.isnan(diagonal1_lengths)
        )

        # TODO check if we need to do something about surrounding vertices
        # add one extra colum and row for contour and contourf

        if (to_mask.shape[0] == Z.shape[0] - 1 and
                to_mask.shape[1] == Z.shape[1] - 1):
            to_mask_extended = np.zeros(Z.shape, dtype=bool)
            to_mask_extended[:-1, :-1] = to_mask
            to_mask_extended[-1, :] = to_mask_extended[-2, :]
            to_mask_extended[:, -1] = to_mask_extended[:, -2]
            to_mask = to_mask_extended

        if np.any(to_mask):
            Z_mask = getattr(Z, 'mask', None)
            to_mask = to_mask if Z_mask is None else to_mask | Z_mask
            Z = np.ma.masked_where(to_mask, Z)

        return ptx, pty, Z


In [32]:
SST_new = SST[:-1, :-1]

In [67]:
plt.figure()
ax1 = plt.subplot(1,1,1,projection=ccrs.PlateCarree(central_longitude=0)) #


#-- Compute the corrected data to prevent line wrapping. Use the dataset
#-- coordinates and variable.
X, Y, maskedZ = z_masked_overlap(ax1,
                                 lon.data,
                                 lat.data,
                                 SST)
cmap='YlGnBu'

plot = ax1.contourf(X, Y, maskedZ, levels=np.arange(-4,34),cmap=cmap,vmin=0,vmax=30)
plt.colorbar(plot, ax=ax1, shrink=0.6, pad=0.09)

In [48]:
maskedZ

masked_array(
  data=[[0.0, 0.0, 0.0, ..., --, --, --],
        [0.0, 0.0, 0.0, ..., --, --, --],
        [0.0, 0.0, 0.0, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float32)